## Final Project D

<a id="item31"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>

<a id='item32'></a>

#### Let's go ahead and import the Keras library

In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

## Change the DL Model to have 3 hidden layers as below

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


<a id="item4"></a>

<a id='item34'></a>

## Train and Test the Network

Let's call the function now to create our model.

In [14]:
# build the model
model = regression_model()

In [15]:
from sklearn.model_selection import train_test_split

## Taking in a Normalized set of features

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=200)

In [17]:
# fit the model
epochs = 50
model.fit(X_train, Y_train, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 5s 6ms/step - loss: 1578.1103
Epoch 2/50
721/721 [==============================] - 3s 4ms/step - loss: 1553.4225
Epoch 3/50
721/721 [==============================] - 2s 3ms/step - loss: 1521.5602
Epoch 4/50
721/721 [==============================] - 2s 3ms/step - loss: 1476.0624
Epoch 5/50
721/721 [==============================] - 2s 3ms/step - loss: 1410.1060
Epoch 6/50
721/721 [==============================] - 2s 3ms/step - loss: 1316.0339
Epoch 7/50
721/721 [==============================] - 3s 4ms/step - loss: 1187.6569
Epoch 8/50
721/721 [==============================] - 3s 4ms/step - loss: 1022.0480
Epoch 9/50
721/721 [==============================] - 2s 3ms/step - loss: 827.6575
Epoch 10/50
721/721 [==============================] - 2s 3ms/step - loss: 627.0158
Epoch 11/50
721/721 [==============================] - 2s 3ms/step - loss: 450.6085
Epoch 12/50
721/721 [==============================] - 2s 3ms/step - loss: 33

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.160421). Check your callbacks.
  % delta_t_median)


721/721 [==============================] - 3s 4ms/step - loss: 160.7774
Epoch 33/50
721/721 [==============================] - 2s 3ms/step - loss: 159.4259
Epoch 34/50
721/721 [==============================] - 2s 3ms/step - loss: 157.0465
Epoch 35/50
721/721 [==============================] - 2s 3ms/step - loss: 154.7568
Epoch 36/50
721/721 [==============================] - 2s 3ms/step - loss: 152.7440
Epoch 37/50
721/721 [==============================] - 2s 3ms/step - loss: 150.5477
Epoch 38/50
721/721 [==============================] - 3s 4ms/step - loss: 148.8812
Epoch 39/50
721/721 [==============================] - 2s 3ms/step - loss: 147.3678
Epoch 40/50
721/721 [==============================] - 2s 3ms/step - loss: 145.6581
Epoch 41/50
721/721 [==============================] - 2s 3ms/step - loss: 143.6414
Epoch 42/50
721/721 [==============================] - 2s 3ms/step - loss: 141.9283
Epoch 43/50
721/721 [==============================] - 2s 3ms/step - loss: 140.6238
Epoc

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.118833). Check your callbacks.
  % delta_t_median)


721/721 [==============================] - 2s 3ms/step - loss: 139.3460
Epoch 45/50
721/721 [==============================] - 2s 2ms/step - loss: 137.9391
Epoch 46/50
721/721 [==============================] - 2s 3ms/step - loss: 136.3534
Epoch 47/50
721/721 [==============================] - 2s 3ms/step - loss: 135.2658
Epoch 48/50
721/721 [==============================] - 2s 3ms/step - loss: 133.8283
Epoch 49/50
721/721 [==============================] - 3s 4ms/step - loss: 132.9572
Epoch 50/50
721/721 [==============================] - 2s 3ms/step - loss: 131.5211


In [18]:
loss_val = model.evaluate(X_test, Y_test)
Y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 1s 2ms/step


123.16549075228497

In [19]:
#Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.
from sklearn.metrics import mean_squared_error

In [20]:
mse = mean_squared_error(Y_test, Y_pred)
mean = np.mean(mse)
standard_deviation = np.std(mse)
print(mean, standard_deviation)

123.1654888268355 0.0


In [22]:
total_mses = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mses):
    X_train, X_test, Y_train, Y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, Y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, Y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    Y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(Y_test, Y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mses) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 46.727239293959535
MSE 2: 68.74419845037862
MSE 3: 43.134512978464265
MSE 4: 43.6130836388054
MSE 5: 41.77846143160823
MSE 6: 43.706822021879425
MSE 7: 48.20453080853212
MSE 8: 42.35329197757066
MSE 9: 40.62459880248628
MSE 10: 44.85329513796711
MSE 11: 41.27895798420829
MSE 12: 37.12490287953596
MSE 13: 46.15054200922401
MSE 14: 45.25729307156165
MSE 15: 38.70070806213182
MSE 16: 37.06699012089702
MSE 17: 38.664777916226186
MSE 18: 40.757303330504776
MSE 19: 37.258073640101166
MSE 20: 46.476539827859135
MSE 21: 39.534729152049835
MSE 22: 39.02772034333362
MSE 23: 32.77125233894027
MSE 24: 39.158622260232576
MSE 25: 48.89966015368218
MSE 26: 44.76890388661604
MSE 27: 42.065321764899686
MSE 28: 35.67147065443514
MSE 29: 46.339809479451105
MSE 30: 42.44848480965327
MSE 31: 38.98674737134026
MSE 32: 36.793949855569885
MSE 33: 38.82485882904152
MSE 34: 41.69950039564213
MSE 35: 40.972904612717116
MSE 36: 47.692618187963
MSE 37: 41.70070706216263
MSE 38: 42.504731545556325
MSE 39: 37

## How does the mean of the mean squared errors compare to that from Step B?

 ## Mean MSE is quite smaller, by a difference of -72, than step B. The Std Dev is smaller as well. 